In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')
from sklearn.preprocessing import scale, minmax_scale
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
os.listdir('../input/titanic')

# Data Feilds

| Variable | Definition                                 | Key                                            |
|----------|--------------------------------------------|------------------------------------------------|
| Survived | Survival                                   | 0 = No, 1 = Yes                                |
| Pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| Sex      | Sex                                        |                                                |
| Age      | Age in years                               |                                                |
| Sibsp    | # of siblings / spouses aboard the Titanic |                                                |
| Parch    | # of parents / children aboard the Titanic |                                                |
| Ticket   | Ticket number                              |                                                |
| Fare     | Passenger fare                             |                                                |
| Cabin    | Cabin number                               |                                                |
| Embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |

## Data load

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
sub = pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
print('train.shape :', train.shape)
train.head()

In [ ]:
print('test.shape :', test.shape)
test.head()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
train.info()

# Data & EDA

In [ ]:
plt.figure(figsize=(15,7))
sns.countplot(train['Survived']).set(title='count survived (0 = dead, 1 = survived)')

In [ ]:
print('count')
print(train['Survived'].value_counts())
print('------------------------------')
print('percentage')
print(train['Survived'].value_counts()/len(train))

죽은 사람들이 더 많다.

In [ ]:
print('survived.mean by Pclass')
train.groupby('Pclass')['Survived'].mean().reset_index()

Pclass에 따른 생존률의 차이가 큼.

In [ ]:
print('survived.mean by Sex')
train.groupby('Sex')['Survived'].mean().reset_index()

Sex에 따른 생존률 차이가 매우 큼.  
여성의 생존률이 훨씬 높음.

In [ ]:
print('survived.mean by SibSp')
train.groupby('SibSp')['Survived'].mean().reset_index()

혼자 탑승한 경우 생존률이 낮고  
형제, 배우자와 같이 탑승한 경우에는 동승자가 많아질수록 생존률 하락.

In [ ]:
print('survived.mean by Parch')
train.groupby('Parch')['Survived'].mean().reset_index()

혼자 탑승한 경우 역시 생존률이 낮고  
부모, 자식과 같이 탑승한 경우에는 동승자가 많아질수록 생존률 하락.  
하지만 동승자가 1,2,3명인 경우 비슷함.

In [ ]:
print('survived.mean by Embarked')
train.groupby('Embarked')['Survived'].mean().reset_index()

Embarked는 C일 때 생존률이 Q와 S보다 높음.  
Q와 S일 때는 비슷.

In [ ]:
def cab(df):
    if type(df)==float:
        return 0
    else:
        return 1

In [ ]:
train['Cabin'] = train['Cabin'].apply(cab)
train.isna().sum()

In [ ]:
test['Cabin'] = test['Cabin'].apply(cab)
test.isna().sum()

In [ ]:
train.groupby('Cabin')['Survived'].mean()

Cabin이 결측값인 사람과 아닌사람의 생존률 차이가 크다.

In [ ]:
plt.figure(figsize=(15,7))
sns.kdeplot(train[train['Survived']==1]['Age'], label='survived')
sns.kdeplot(train[train['Survived']==0]['Age'], label='died').set(title='age dist. by survival',
                                                                  xlabel='Age',
                                                                  ylabel='density')

아이들의 생존률이 높아보인다.  
10대 후반 ~ 20대는 죽은 사람들이 더 많다.

In [ ]:
plt.figure(figsize=(15,7))
sns.kdeplot(train[train['Survived']==1]['Fare'], label='survived')
sns.kdeplot(train[train['Survived']==0]['Fare'], label='died').set(title='fare dist. by survival',
                                                                   xlabel='Fare',
                                                                   ylabel='density')

돈을 적게 낸 사람들의 생존률은 현저히 낮아보인다.

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(train['Age'], bins=20)
plt.xlabel('Age')
plt.ylabel('count')
plt.title('Age dist.')

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(test['Age'], bins=20)
plt.xlabel('Age')
plt.ylabel('count')
plt.title('Age dist.')

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(train['Fare'], bins=40)
plt.xlabel('fare')
plt.ylabel('count')
plt.title('train Fare dist.')

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(test['Fare'], bins=40)
plt.xlabel('fare')
plt.ylabel('count')
plt.title('test Fare dist.')

Train set과 Test set의 Age와 Fare의 분포가 비슷하므로 결측값처리시 합쳐서 처리해도 무방하다고 판단.

## missing value
Embarked는 최빈값으로 대체  
Fare는 중앙값으로 대체  
Cabin은 있다 없다로 나누었음(0=없다, 1=있다)  
Age는 중앙값으로 대체  
(근데 Age는 Name과 Sex를 사용해 결측값 처리하는 것이 제일 나아보이긴 함.)

In [ ]:
data = pd.concat([train,test])
data.head()

In [ ]:
data['Embarked'].value_counts()

In [ ]:
data['Fare'].median()

In [ ]:
data['Age'].median()

In [ ]:
data['Embarked'] = data['Embarked'].fillna('S')
data['Fare'] = data['Fare'].fillna(data['Fare'].median())
data['Age'] = data['Age'].fillna(data['Age'].median())
data.isna().sum()

In [ ]:
plt.figure(figsize=(15,7))
sns.boxplot(x='Pclass', y='Age', data=data)

Age는 Pclass에 따라 중앙값의 차이가 있지만 전체 중앙값을 넣어도 별상관이 없다고 생각.

In [ ]:
data.drop(['Name','Ticket'], axis=1, inplace=True)

Name과 Ticket은 사용하지 않을 것이다.

# Feature Engineering

In [ ]:
data.head()

Embarked, Pclass 더미변수 생성.

In [ ]:
ed = pd.get_dummies(data['Embarked'], prefix='Embarked')
ed.head()

In [ ]:
pdu = pd.get_dummies(data['Pclass'], prefix='Pclass')
pdu.head()

In [ ]:
data = pd.concat([data,ed], axis=1, join='inner')
data = pd.concat([data,pdu], axis=1, join='inner')
data.head()

In [ ]:
data.drop(['Embarked','Pclass'], axis=1, inplace=True)
data.head()

label encoding된 변수를 숫자형으로 바꿔줌.

In [ ]:
data['Sex'] = data['Sex'].map({'male':0, 'female':1})
data.head()

Age를 범주화 해줌.  
그 이유는 이게 재난사건이기도 하고, 그래프를 보면 나이대별로 생존률이 확실히 다르기 때문.  
재난이 일어나면 보통 어린아이, 나이든사람, 여성을 먼저 살리려한다.  
그래프를 보고 패턴을 찾아 범주화할 것인데, 보통 이 방법을 선택하진 않지만 재난사건이라는 특수한 경우이기 때문에 시도해보려 한다.

In [ ]:
plt.figure(figsize=(15,7))
sns.kdeplot(data[data['Survived']==1]['Age'], label='survived')
sns.kdeplot(data[data['Survived']==0]['Age'], label='dead').set(title='age dist. by survived',
                                                                xlabel='Age',
                                                                ylabel='density')

Age의 결측값을 중앙값으로 대체해 분포가 좀 바뀐 듯 하다.

In [ ]:
data[data['Age']<=10]['Survived'].mean()

In [ ]:
data[(data['Age']>10)&(data['Age']<=30)]['Survived'].mean()

In [ ]:
data[(data['Age']>31)&(data['Age']<=57)]['Survived'].mean()

In [ ]:
data[data['Age']>57]['Survived'].mean()

In [ ]:
def age_change(x):
    if x<=10:
        return 1
    elif (x>10) & (x<=30):
        return 2
    elif (x>30) & (x<=57):
        return 3
    else:
        return 4

In [ ]:
data['Age'] = data['Age'].map(age_change)
data.head()

In [ ]:
ad = pd.get_dummies(data['Age'], prefix='Age')
ad.head()

In [ ]:
data = pd.concat([data,ad], axis=1, join='inner')
data.drop('Age', axis=1, inplace=True)
data.head()

In [ ]:
plt.figure(figsize=(15,7))
sns.kdeplot(data[data['Survived']==1]['Fare'], label='survived')
sns.kdeplot(data[data['Survived']==0]['Fare'], label='died')

Fare가 적을 때 생존률이 엄청 낮지만 이상치를 잡기위해 그대로 넣을 것이다.

In [ ]:
train = data[data['Survived'].isna()==False]
test = data[data['Survived'].isna()==True]
print(train.shape, test.shape)

In [ ]:
x_train = train.drop(['PassengerId','Survived'], axis=1)
y_train = train['Survived']
x_test = test.drop(['PassengerId','Survived'], axis=1)
x_train.head()

# Modeling

In [ ]:
x_tra, x_val, y_tra, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=19)

In [ ]:
xgb_params = {'n_estimators':5000,
              'learning_rate':0.1,
              'max_depth':3}

In [ ]:
xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model.fit(x_tra, y_tra,
              eval_set = [(x_val, y_val)],
              eval_metric = 'auc',
              early_stopping_rounds = 50,
              verbose = 100)

In [ ]:
pred = xgb_model.predict(x_test, ntree_limit = xgb_model.best_iteration)
pred[:20]

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
xgb.plot_importance(xgb_model, ax=ax)
plt.show()

In [ ]:
sub['Survived'] = pred
sub['Survived'] = sub['Survived'].astype(int)
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)